In [1]:
import numpy as np
import pandas as pd

import torch
from torch.optim import Adam
import torch.nn.functional as F
from sklearn.metrics import accuracy_score

In [2]:
df = pd.read_csv('./raw_data/softmax.txt',sep=' ',header=None)
df

,0,1,2,3,4
0,2,1,0,0,1
1,3,2,0,0,1
2,3,4,0,0,1
3,5,5,0,1,0
4,7,5,0,1,0
5,2,5,0,1,0
6,6,6,1,0,0
7,7,7,1,0,0


In [3]:
x_data = df.iloc[:,:-3]
y_data = df.iloc[:,-3:]

In [4]:
x = torch.FloatTensor(x_data.values)
y = torch.FloatTensor(y_data.values)

w = torch.empty( [2,3], requires_grad=True ) 
b = torch.empty( [3], requires_grad=True)
torch.nn.init.uniform_(w)
torch.nn.init.uniform_(b)

def cost():
    """
    
    hx = F.softmax(z,dim=1)
    cost_i = (y*-torch.log(hx).sum())
    
    """
    
    z = torch.matmul(x,w) + b
    c_i = F.cross_entropy(z,y)
    c = torch.mean(c_i)
    return c

def hxFn(x_data):
    xd = torch.FloatTensor(x_data)
    z = torch.matmul(xd,w) + b
    hx = torch.softmax(z,dim=1)
    return z, hx

In [5]:
optimizer = Adam( [w,b], lr=0.01)
for epoch in range(2000):
    c = cost()
    optimizer.zero_grad()
    c.backward()
    temp=accuracy_score((torch.matmul(x,w)+b).detach().numpy().argmax(axis=1),y.numpy().argmax(axis=1))
    optimizer.step()
    if temp != accuracy_score((torch.matmul(x,w)+b).detach().numpy().argmax(axis=1),y.numpy().argmax(axis=1)):
        print(f'{epoch}','cost', c.item(), 'acc :',accuracy_score((torch.matmul(x,w)+b).detach().numpy().argmax(axis=1),y.numpy().argmax(axis=1)) )

17 cost 1.1531760692596436 acc : 0.625
18 cost 1.1235754489898682 acc : 0.5
21 cost 1.068286657333374 acc : 0.375
22 cost 1.0578426122665405 acc : 0.25
26 cost 1.0387625694274902 acc : 0.375
27 cost 1.0377347469329834 acc : 0.5
28 cost 1.037640929222107 acc : 0.375
29 cost 1.0382407903671265 acc : 0.25
30 cost 1.0392992496490479 acc : 0.375
51 cost 0.9968646764755249 acc : 0.5
55 cost 0.986590564250946 acc : 0.625
57 cost 0.9823665618896484 acc : 0.75
63 cost 0.9718044996261597 acc : 0.625
84 cost 0.9373644590377808 acc : 0.75
118 cost 0.884341835975647 acc : 0.625
177 cost 0.8051815032958984 acc : 0.75
527 cost 0.5707530379295349 acc : 0.875
983 cost 0.4387438893318176 acc : 1.0


In [6]:
hxFn([[1,1],[5,6]])

(tensor([[-6.6162,  2.2457,  8.0953],
         [11.4730, 11.0350,  7.4060]], grad_fn=<AddBackward0>),
 tensor([[4.0703e-07, 2.8728e-03, 9.9713e-01],
         [6.0152e-01, 3.8817e-01, 1.0303e-02]], grad_fn=<SoftmaxBackward0>))

In [8]:
def hxFn(x_data):
    xd = torch.FloatTensor(x_data)
    z = torch.matmul(xd,w) + b
    hx = torch.softmax(z,dim=1)
    return hx.argmax(dim=1).detach().numpy() # torch.argmax(hx,dim=1) <=> np.argmax(hx,axis=1)
hxFn([[1,1],[5,6]])

array([2, 0])